## 핵심 기능을 가진 웹서비스 구현 실습

### 웹서비스 기획에 대한 가벼운 이해

### MVP (Minimum Viable Product)
- 최소한의 기능을 구현한 제품
- 고객의 피드백을 받아서, 기능을 점차적으로 개선
- 최근 스타트업이 개발하는 방식


### 웹서비스 구현을 위해 알아둬야 하는 MVC 패턴
- **M**odel-**V**iew-**C**ontroller : **소프트웨어 공학 학문 분야**에서 나온 아키텍쳐 
  - Model: 응용 프로그램의 데이터를 나타냄 (주로 데이터베이스)
  - View: 텍스트, 버튼등 사용자 인터페이스를 나타냄 (웹페이지)
  - Controller: Model 과 View 를 제어하는 중간 역할 (중간 제어 코드)
- MVC 패턴대로 코드를 작성하면, 유지보수가 쉬워진다는 주장을 한 것임

#### 실제로는 그렇지 않은 경우가 많음
- MVC 패턴은 하나의 기능을 구현하는데 각 기능이 여러 파일에 쪼개져 있으므로, 코드 이해/디버깅이 어려움
- 코드 수정시, 결국 MVC 특정 부분만 수정하기 보다, 전체 연결된 기능 관련 코드를 수정하는 경우가 더 많아지는 경우도 많음

> 기존 코드가 MVC 패턴이면, 기존 코드 구조에 맞게 작성, 그렇지 않으면 참고만 하세요



------------------

### MVC 패턴과 flask blueprint

- 한 파일에 모든 기능 코드를 넣으면, 복잡해지고, 관리가 어려움
- re-usability 를 항상 생각해야 함
  - 다만, 그렇다고 대형 프로그램에서나 필요한 구조를 가져갈 필요는 없음

    ### flask 백엔드 코드 구조
- 기능별로 폴더/파일 구분 (blueprint 를 사용해서, 기능별로 추가/삭제가 쉽도록 구성)
- MVC 에서 View 에 해당하는 부분은 없음 (순수 Rest API 서버로, 추후 해당 API 를 사용한 프론트앤드 별도 구축)
- C 는 API, M 은 데이터베이스/데이터 모델 로만 구성하면 됨



    ### 기능별로 폴더를 나눠서 코드 구현하기로 함
- 간단한 코드일 경우, MVC 패턴을 사용하면, 코드 구현 및 확인을 위해 여러 소스파일을 참고해야 하므로 오히려 시간이 많이 걸림
  - MVC 패턴은 매우 복잡한 기능을 가진 큰 규모의 시스템에서, 일부 유용할 수 있음
- 하지만, 기본적인 MVC 패턴 이해를 위해, 다음과 같이 구성
  - flask 서버: blog_abtest.py 
  - <b>M</b>odel: db_model
  - <b>V</b>iew: blog_view
  - <b>C</b>ontrol: blog_control
  - 이외에 
    - static 은 부트스트랩등 HTML 파일에서 필요로 하는 동일 서버 내의 javascript 와 css 파일
    - templates 는 flask 서버가 지원하는 HTML 파일  
> 구현한 폴더 구조 Control,View,dbModel 로 나눠서 작성
```bash
.
├── blog_abtest.py
├── blog_control
│   ├── session_mgmt.py
│   └── user_mgmt.py
├── blog_view
│   └── blog.py
├── db_model
│   ├── mongodb.py
│   └── mysql.py
├── schema.sql
├── static
│   ├── blog.css
│   ├── brand
│   └── dist
└── templates
    ├── blog_A.html
    └── blog_B.html
```

    ### flask blueprint 기능을 활용하여 본 서버파일에서 라우팅경로까지 작성하지 않도록한다!
- 여러 소스 파일에 flask 코드를 작성할 수 있도록 하는 기능

- 메인 코드 파일 내

```python
from flask import Flask
from 하위폴더명(폴더명이 blog_view 이면 blog_view) import 하위폴더의소스파일명(blog.py 이면 blog)

app = Flask(__name__)
app.register_blueprint(하위폴더의소스파일명.블루프린트객체이름, url_prefix='/blog')
```
- url_prefix=기본경로명
  - 하위폴더의 소스파일에 있는 라우팅 경로는 URL/기본경로명/라우팅 경로 와 같이 설정됨

- 분리된 하위 폴더의 소스 파일 내
  - Blueprint(블루프린트이름, __name__) 으로 작성
  
```python
from flask import Blueprint
# 블루프린트객체이름은 blog_abtest 가 됨
blog_abtest = Blueprint('blog', __name__)

# http://localhost:8080/blog/blog1
@blog_abtest.route('/blog1')
def blog():
    return "blueprint test login_google"
```

### 예제

In [ ]:
# dave_server.py 본서버 
from flask import Flask
from test_view import blog_view

app = Flask(__name__)
app.register_blueprint(blog_view.blog_ab, url_prefix='/blog')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port='8080')

In [ ]:
# blog.py 블루프린트 활용한 하위폴더 라우팅경로 정의 파일
from flask import Blueprint

blog_ab = Blueprint('blog', __name__)

# http://localhost:8080/blog/blog1
@blog_ab.route('/blog1')
def blog():
    return 'TEST BLUEPRINT'

---------------
# 이러한 과정을 거쳐서...

### ABTest기능 웹서비스 구현 목표
- 블로그 사이트를 구현 예정
  - 블로그 페이지를 웹으로 오픈
  - ABTest 로 고객에게 다른 페이지를 보여주는 기능
  - 각 페이지마다 고객 이메일 구독 유도 기능
  - 이메일 구독시, 이메일 저장 기능
  - 이메일 구독시, 고객에게 이메일 구독중임을 알려주는 기능
  - 각 페이지에 얼마나 많은 사람들이 접근했는지 접근 시간 저장 기능

### 구현할 기능
- flask 로 프론트엔드와 백엔드를 모두 구현
- 블로그 페이지 제목만 변경하여, 두 가지 버전 구현
- 동일한 라우팅 경로 접속시마다 두 가지 버전 중 하나가 보여지도록 함 (A: 50%, B: 50%)
  - 접속 IP 와 접속시간을 로그로 남김
- 구독시 이메일 주소를 받아서 저장함
  - 해당 사용자는 flask 서버가 재실행되지 않는 이상, 동일 블로그 페이지가 보여지고, 구독창이 더이상 뜨지 않도록 함
  
> 이런 서비스 구현 후, 많은 사용자를 기반으로 실험해본다면, 어떤 블로그 제목이 더 구독자를 많이 끌어모을 수 있는지를 로그 분석을 기반으로 알 수 있음 <br>
> 라우팅 경로를 무한히 만들어, 다양한 블로그 서비스 가능